In [1]:

from pyspark.sql import SparkSession

from awsglue.context import GlueContext
spark = SparkSession.builder \
    .appName("Glue-Iceberg-Notebook") \
    .config("spark.sql.catalog.glue_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.profile.ProfileCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.profile", "default") \
    .config("spark.driver.extraJavaOptions", 
            "-Dcom.amazonaws.sdk.disableEc2Metadata=true "
            "-Daws.region=ap-southeast-1 ") \
    .config("spark.sql.catalog.glue_catalog.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config("spark.sql.catalog.glue_catalog.warehouse", "s3://scm-analytics-2025/iceberg/bronze/warehouse/") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.jars.packages","mysql:mysql-connector-java:8.0.33,") .getOrCreate()
sc = spark.sparkContext
glueContext = GlueContext(sc)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-38fe9742-6466-4f91-bfc5-e222b4847d4e;1.0
	confs: [default]
mysql#mysql-connector-java;8.0.33 is relocated to com.mysql#mysql-connector-j;8.0.33. Please update your dependencies.
	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
downloading https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/8.0.33/mysql-connector-j-8.0.33.jar ...
	[SUCCESSFUL ] com.mysql#mysql-connector-j;8.0.33!mysql-connector-j.jar (363ms)
downloading https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/3.21.9/protobuf-java-3.21.9.jar ...
	[SUCCESSFUL ] com.google.protobuf#protobuf-java;3.21.9!protobuf-java.jar(bundle) (351ms)
:: resolution report :: resolve

In [3]:
query = f"""


SELECT 'scm_silver.dim_customer', COUNT(*) FROM glue_catalog.scm_silver.dim_customer
UNION ALL
SELECT 'scm_silver.dim_product', COUNT(*) FROM glue_catalog.scm_silver.dim_product
UNION ALL
SELECT 'scm_silver.dim_supplier', COUNT(*) FROM glue_catalog.scm_silver.dim_supplier
UNION ALL
SELECT 'scm_silver.dim_warehouse', COUNT(*) FROM glue_catalog.scm_silver.dim_warehouse
UNION ALL
SELECT 'scm_silver.fact_inventory', COUNT(*) FROM glue_catalog.scm_silver.fact_inventory
UNION ALL
SELECT 'scm_silver.fact_sales_order', COUNT(*) FROM glue_catalog.scm_silver.fact_sales_order
UNION ALL
SELECT 'scm_silver.fact_shipments', COUNT(*) FROM glue_catalog.scm_silver.fact_shipments;



"""
spark.sql(query).show(truncate=False)

+---------------------------+--------+
|scm_silver.dim_customer    |count(1)|
+---------------------------+--------+
|scm_silver.dim_customer    |101     |
|scm_silver.dim_product     |149     |
|scm_silver.dim_supplier    |576     |
|scm_silver.dim_warehouse   |148     |
|scm_silver.fact_inventory  |4464    |
|scm_silver.fact_sales_order|48567   |
|scm_silver.fact_shipments  |29434   |
+---------------------------+--------+



In [4]:
query = f"""
SELECT 'scm_gold.fact_sales_summary' AS table_name, COUNT(*) AS row_count FROM glue_catalog.scm_gold.fact_sales_summary
UNION ALL
SELECT 'scm_gold.fact_sales_summary_by_country', COUNT(*) FROM glue_catalog.scm_gold.fact_sales_summary_by_country
UNION ALL
SELECT 'scm_gold.fact_sales_summary_by_supplier_daily', COUNT(*) FROM glue_catalog.scm_gold.fact_sales_summary_by_supplier_daily
UNION ALL
SELECT 'scm_gold.fact_sales_summary_by_supplier_monthly', COUNT(*) FROM glue_catalog.scm_gold.fact_sales_summary_by_supplier_monthly
UNION ALL
SELECT 'scm_gold.fact_supply_chain_summary', COUNT(*) FROM glue_catalog.scm_gold.fact_supply_chain_summary
UNION ALL
SELECT 'scm_gold.high_shipping_orders', COUNT(*) FROM glue_catalog.scm_gold.high_shipping_orders
UNION ALL
SELECT 'scm_gold.inventory_overstock_inactive', COUNT(*) FROM glue_catalog.scm_gold.inventory_overstock_inactive
UNION ALL
SELECT 'scm_gold.late_deliveries_daily', COUNT(*) FROM glue_catalog.scm_gold.late_deliveries_daily
UNION ALL
SELECT 'scm_gold.order_stock_shortage', COUNT(*) FROM glue_catalog.scm_gold.order_stock_shortage
UNION ALL
SELECT 'scm_gold.sales_order_lead_time', COUNT(*) FROM glue_catalog.scm_gold.sales_order_lead_time
UNION ALL
SELECT 'scm_gold.supplier_delivery_performance', COUNT(*) FROM glue_catalog.scm_gold.supplier_delivery_performance



"""
spark.sql(query).show(truncate=False)

+-----------------------------------------------+---------+
|table_name                                     |row_count|
+-----------------------------------------------+---------+
|scm_gold.fact_sales_summary                    |0        |
|scm_gold.fact_sales_summary_by_country         |0        |
|scm_gold.fact_sales_summary_by_supplier_daily  |0        |
|scm_gold.fact_sales_summary_by_supplier_monthly|0        |
|scm_gold.fact_supply_chain_summary             |0        |
|scm_gold.high_shipping_orders                  |0        |
|scm_gold.inventory_overstock_inactive          |0        |
|scm_gold.late_deliveries_daily                 |0        |
|scm_gold.order_stock_shortage                  |0        |
|scm_gold.sales_order_lead_time                 |0        |
|scm_gold.supplier_delivery_performance         |0        |
+-----------------------------------------------+---------+



In [ ]:
spark.sql("SELECT customer_name,region,country,start_date,end_date,is_current FROM glue_catalog.scm_silver.dim_customer where customer_id ='29951' ").show()